<a href="https://colab.research.google.com/github/H-y-hoon/2023_LG_DISPLAY_Quality_Classification/blob/main/Team_KNU_online.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **스마트 공장 제품 품질 상태 분류 AI 온라인 해커톤**

---



[주제]

스마트 공장의 제어 시스템 구축을 위한 제품 품질 분류 AI 모델 개발

# 코랩 사용으로 마운트와 연결

In [ ]:
#구글 드라이브와 mount(연결)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 모델 다운로드

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.2 MB/s eta 0:00:00


#라이브러리 호출

In [ ]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier

#데이터 로드

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Lg_aimers_Hackathon/data/train.csv', encoding = 'utf-8')
test = pd.read_csv('/content/drive/MyDrive/Lg_aimers_Hackathon/data/test.csv', encoding = 'utf-8')

#데이터 탐색

In [ ]:
#train데이터 확인
train.head()

,PRODUCT_ID,Y_Class,Y_Quality,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TRAIN_000,1,0.533433,2022-06-13 5:14,T050304,A_31,NaN,NaN,NaN,NaN,...,39.34,40.89,32.56,34.09,77.77,NaN,NaN,NaN,NaN,NaN
1,TRAIN_001,2,0.541819,2022-06-13 5:22,T050307,A_31,NaN,NaN,NaN,NaN,...,38.89,42.82,43.92,35.34,72.55,NaN,NaN,NaN,NaN,NaN
2,TRAIN_002,1,0.531267,2022-06-13 5:30,T050304,A_31,NaN,NaN,NaN,NaN,...,39.19,36.65,42.47,36.53,78.35,NaN,NaN,NaN,NaN,NaN
3,TRAIN_003,2,0.537325,2022-06-13 5:39,T050307,A_31,NaN,NaN,NaN,NaN,...,37.74,39.17,52.17,30.58,71.78,NaN,NaN,NaN,NaN,NaN
4,TRAIN_004,1,0.531590,2022-06-13 5:47,T050304,A_31,NaN,NaN,NaN,NaN,...,38.70,41.89,46.93,33.09,76.97,NaN,NaN,NaN,NaN,NaN


In [ ]:
#test데이터 확인
test.head()

,PRODUCT_ID,TIMESTAMP,LINE,PRODUCT_CODE,X_1,X_2,X_3,X_4,X_5,X_6,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
0,TEST_000,2022-09-09 2:01,T100306,T_31,2.0,94.0,0.0,45.0,10.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TEST_001,2022-09-09 2:09,T100304,T_31,2.0,93.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TEST_002,2022-09-09 8:42,T100304,T_31,2.0,95.0,0.0,45.0,11.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TEST_003,2022-09-09 10:56,T010305,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TEST_004,2022-09-09 11:04,T010306,A_31,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
train.shape

(598, 2881)

In [ ]:
test.shape

(310, 2879)

In [ ]:
#train데이터 결측치 확인
train.isnull().sum()

PRODUCT_ID      0
Y_Class         0
Y_Quality       0
TIMESTAMP       0
LINE            0
             ... 
X_2871        499
X_2872        598
X_2873        598
X_2874        598
X_2875        598
Length: 2881, dtype: int64

In [ ]:
#test데이터 결측치 확인
test.isnull().sum()

PRODUCT_ID        0
TIMESTAMP         0
LINE              0
PRODUCT_CODE      0
X_1              67
               ... 
X_2871          271
X_2872          301
X_2873          301
X_2874          301
X_2875          301
Length: 2879, dtype: int64

In [ ]:
#train데이터 기초통계량 확인
train.describe()

,Y_Class,Y_Quality,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
count,598.000000,598.000000,349.000000,349.000000,349.0,349.0,349.000000,349.0,349.000000,349.000000,...,100.000000,100.00000,100.000000,100.000000,100.00000,99.0,0.0,0.0,0.0,0.0
mean,1.025084,0.530896,2.409742,95.123209,0.0,45.0,10.392550,0.0,48.802292,10.048711,...,50.807300,53.60770,49.606200,51.659800,66.64970,1.0,NaN,NaN,NaN,NaN
std,0.565069,0.007401,5.895256,4.107640,0.0,0.0,0.489019,0.0,4.373824,0.215571,...,7.011828,8.13899,7.158917,8.913065,4.52781,0.0,NaN,NaN,NaN,NaN
min,0.000000,0.500856,1.000000,87.000000,0.0,45.0,10.000000,0.0,45.000000,10.000000,...,32.120000,31.70000,32.560000,30.490000,61.67000,1.0,NaN,NaN,NaN,NaN
25%,1.000000,0.527535,2.000000,93.000000,0.0,45.0,10.000000,0.0,45.000000,10.000000,...,49.485000,52.20000,42.160000,49.915000,63.64500,1.0,NaN,NaN,NaN,NaN
50%,1.000000,0.530436,2.000000,95.000000,0.0,45.0,10.000000,0.0,45.000000,10.000000,...,53.425000,55.92500,51.460000,56.175000,65.14000,1.0,NaN,NaN,NaN,NaN
75%,1.000000,0.533433,2.000000,98.000000,0.0,45.0,11.000000,0.0,51.000000,10.000000,...,55.287500,58.97500,55.030000,57.175000,67.11500,1.0,NaN,NaN,NaN,NaN
max,2.000000,0.578841,103.000000,102.000000,0.0,45.0,11.000000,0.0,62.000000,11.000000,...,60.240000,68.66000,60.410000,59.930000,79.75000,1.0,NaN,NaN,NaN,NaN


In [ ]:
#test데이터 기초통계량 확인
test.describe()

,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,X_9,X_10,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
count,243.000000,243.000000,243.0,243.0,243.000000,243.0,243.000000,243.000000,243.000000,243.0,...,34.000000,34.000000,34.000000,34.000000,34.000000,39.000000,9.000000,9.000000,9.000000,9.000000
mean,4.337449,94.448560,0.0,45.0,10.271605,0.0,48.485597,10.012346,42.407407,2.0,...,52.066471,53.014706,49.994412,54.202353,63.806176,0.974359,247.988889,251.888889,243.300000,228.844444
std,16.624229,4.062582,0.0,0.0,0.445705,0.0,3.733199,0.110651,10.482308,0.0,...,2.715660,4.431279,6.116252,3.419513,1.924128,0.160128,25.467305,20.078872,35.523689,19.514233
min,1.000000,87.000000,0.0,45.0,10.000000,0.0,45.000000,10.000000,31.000000,2.0,...,44.410000,41.630000,39.850000,47.820000,60.720000,0.000000,199.000000,214.600000,169.700000,191.900000
25%,2.000000,91.000000,0.0,45.0,10.000000,0.0,45.000000,10.000000,31.000000,2.0,...,51.237500,50.095000,43.750000,52.170000,62.295000,1.000000,247.100000,253.900000,231.100000,232.800000
50%,2.000000,95.000000,0.0,45.0,10.000000,0.0,50.000000,10.000000,52.000000,2.0,...,51.810000,53.655000,51.175000,55.605000,63.170000,1.000000,258.700000,257.500000,261.700000,236.300000
75%,2.000000,97.000000,0.0,45.0,11.000000,0.0,51.000000,10.000000,52.000000,2.0,...,53.457500,55.957500,54.097500,56.642500,65.442500,1.000000,263.400000,261.300000,263.200000,239.700000
max,154.000000,102.000000,0.0,45.0,11.000000,0.0,67.000000,11.000000,52.000000,2.0,...,58.060000,60.490000,60.310000,58.290000,67.240000,1.000000,268.000000,278.600000,271.400000,245.600000


#데이터 전처리

In [ ]:
train_x = train.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train['Y_Class']

test_x = test.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [ ]:
#결측치 제거
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [ ]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


#모델링


In [ ]:
#CatBoost 사용. random_state는 325로 지정.
Cat = CatBoostClassifier(random_state=325).fit(train_x, train_y)
print('Done.')

Learning rate set to 0.077079
0:	learn: 1.0449328	total: 735ms	remaining: 12m 14s
1:	learn: 1.0002539	total: 1.11s	remaining: 9m 13s
2:	learn: 0.9625919	total: 1.46s	remaining: 8m 5s
3:	learn: 0.9280810	total: 1.84s	remaining: 7m 37s
4:	learn: 0.9011416	total: 2.21s	remaining: 7m 19s
5:	learn: 0.8780688	total: 2.58s	remaining: 7m 7s
6:	learn: 0.8578244	total: 2.94s	remaining: 6m 56s
7:	learn: 0.8375480	total: 3.3s	remaining: 6m 48s
8:	learn: 0.8207219	total: 3.67s	remaining: 6m 44s
9:	learn: 0.8056250	total: 4.04s	remaining: 6m 39s
10:	learn: 0.7928107	total: 4.41s	remaining: 6m 36s
11:	learn: 0.7810962	total: 4.78s	remaining: 6m 33s
12:	learn: 0.7694733	total: 5.13s	remaining: 6m 29s
13:	learn: 0.7583369	total: 5.49s	remaining: 6m 26s
14:	learn: 0.7481232	total: 5.86s	remaining: 6m 24s
15:	learn: 0.7358691	total: 6.22s	remaining: 6m 22s
16:	learn: 0.7280341	total: 6.58s	remaining: 6m 20s
17:	learn: 0.7192274	total: 6.97s	remaining: 6m 20s
18:	learn: 0.7132208	total: 7.4s	remaining: 6m

#데이터 추론

In [ ]:
preds = Cat.predict(test_x)
print('Done.')

Done.


#결과값 저장

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Lg_aimers_Hackathon/data/sample_submission.csv')

In [ ]:
submission['Y_Class'] = preds

In [ ]:
submission.to_csv('./cat_submission11.csv', index=False)